In [1]:
from w2v_utils import *
import pandas as pd 
import os
from bs4 import BeautifulSoup 
# The package comes built-in with Python
import re
# http://www.nltk.org/install.html
import nltk
from nltk.corpus import stopwords # Import the stop word list

Using TensorFlow backend.


In [2]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import plot_model

## Functions

In [3]:
def text_to_words(raw_text):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_text, "lxml").get_text() 

    # 2. Remove non-letters
    # Find anything that is NOT a lowercase letter (a-z) or an upper case letter (A-Z), and replace it with a space
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    # stops = set(stopwords.words("english"))
    
    # There are many other things we could do to the data
    # For example, Porter Stemming and Lemmatizing (both available in NLTK) 
    # would allow us to treat "messages", "message", and "messaging" as the same word
    # which could certainly be useful.
    
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    clear_text = " ".join(meaningful_words)
    return clear_text
    #validate_text_sentiment["Message"][i] = clear_text 
    #print i
    #return( " ".join( meaningful_words ))  

In [4]:
# GRADED FUNCTION: sentence_to_avg

def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    ### START CODE HERE ###
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = [i.lower() for i in sentence.split()]
    
    # Remove unknown words
    cleaned_words = []
    for word in words:
        if word not in errors:
            cleaned_words.append(word)
            
    words = cleaned_words
        
    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros((50,))
    
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    for w in words:
        avg += word_to_vec_map[w]
    avg = avg / len(words)
    
    ### END CODE HERE ###
    
    return avg

In [5]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

## Import data

In [6]:
df_train = pd.read_csv("Data/task_train.csv")

In [7]:
df_test = pd.read_csv("Data/task_test.csv")

In [8]:
words, word_to_vec_map = read_glove_vecs('Data/glove.6B.50d.txt')

## Clean and preprocess

In [13]:
stops = set(stopwords.words("english"))

In [14]:
df_train['cleaned_text'] = df_train.apply (lambda row: text_to_words(row.review), axis=1)

In [15]:
df_train = df_test

In [16]:
merged = []
for review in df_train['cleaned_text']:
    merged = merged + review.split()

In [17]:
len(merged)

303347

In [18]:
vocabulary = np.unique(np.array(merged))

In [19]:
errors = []

for word in vocabulary:
    try:
        word_to_vec_map[word]
    except:
        errors.append(word)

In [20]:
len(errors)

1844

In [21]:
data = []
for review in df_train['cleaned_text']:
    data.append(sentence_to_avg(review, word_to_vec_map))

In [134]:
X = np.array(data)
y = df_train['sentiment']
y = to_categorical(y)

In [135]:
y

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [138]:
X_train.shape

(7860, 50)

In [139]:
y_train.shape

(7860, 2)

## Model

In [26]:
model = Sequential()
model.add(Dense(32, input_dim=50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [27]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
model.fit(X_train, y_train, epochs=100, batch_size=512, validation_split=0.20)

Train on 6288 samples, validate on 1572 samples
Epoch 1/100
6288/6288 [==============================] - 0s 8us/step - loss: 0.4835 - acc: 0.7882 - val_loss: 0.4729 - val_acc: 0.7805
Epoch 2/100
6288/6288 [==============================] - 0s 9us/step - loss: 0.4863 - acc: 0.7848 - val_loss: 0.4713 - val_acc: 0.7799
Epoch 3/100
6288/6288 [==============================] - 0s 8us/step - loss: 0.4885 - acc: 0.7801 - val_loss: 0.4699 - val_acc: 0.7863
Epoch 4/100
6288/6288 [==============================] - 0s 8us/step - loss: 0.4822 - acc: 0.7842 - val_loss: 0.4699 - val_acc: 0.7818
Epoch 5/100
6288/6288 [==============================] - 0s 8us/step - loss: 0.4797 - acc: 0.7850 - val_loss: 0.4703 - val_acc: 0.7786
Epoch 6/100
6288/6288 [==============================] - 0s 8us/step - loss: 0.4793 - acc: 0.7880 - val_loss: 0.4696 - val_acc: 0.7754
Epoch 7/100
6288/6288 [==============================] - 0s 8us/step - loss: 0.4756 - acc: 0.7864 - val_loss: 0.4696 - val_acc: 0.7793
Epoch 8

Epoch 61/100
6288/6288 [==============================] - 0s 7us/step - loss: 0.4758 - acc: 0.7810 - val_loss: 0.4729 - val_acc: 0.7767
Epoch 62/100
6288/6288 [==============================] - 0s 12us/step - loss: 0.4786 - acc: 0.7851 - val_loss: 0.4745 - val_acc: 0.7774
Epoch 63/100
6288/6288 [==============================] - 0s 13us/step - loss: 0.4872 - acc: 0.7858 - val_loss: 0.4738 - val_acc: 0.7786
Epoch 64/100
6288/6288 [==============================] - 0s 11us/step - loss: 0.4804 - acc: 0.7824 - val_loss: 0.4738 - val_acc: 0.7761
Epoch 65/100
6288/6288 [==============================] - 0s 8us/step - loss: 0.4782 - acc: 0.7890 - val_loss: 0.4731 - val_acc: 0.7799
Epoch 66/100
6288/6288 [==============================] - 0s 7us/step - loss: 0.4821 - acc: 0.7832 - val_loss: 0.4732 - val_acc: 0.7761
Epoch 67/100
6288/6288 [==============================] - 0s 7us/step - loss: 0.4815 - acc: 0.7831 - val_loss: 0.4738 - val_acc: 0.7754
Epoch 68/100
6288/6288 [=====================

In [63]:
model.save_weights('model2.h5')

In [38]:
#get TRAINING loss and acc
print("Evaluating training data")
print("====================================================================")
train_score = model.evaluate(X_train, y_train)
print("Test accuracy = ", train_score[1]) 
print("====================================================================")

print()
#get TEST loss and acc
print("Evaluating testing data")
print("====================================================================")
test_score = model.evaluate(X_test, y_test)
Y_test_hat = np.argmax(model.predict(X_test),axis=1)
print("Test accuracy = ", test_score[1]) 
print("====================================================================")

Evaluating training data
7860/7860 [==============================] - 0s 29us/step
Test accuracy =  0.795292620865

Evaluating testing data
1966/1966 [==============================] - 0s 29us/step
Test accuracy =  0.763987792533


In [39]:
from sklearn.metrics import f1_score
print('F1-score of the algorithm is %f' %(f1_score(np.argmax(y_test,axis=1), Y_test_hat)))

F1-score of the algorithm is 0.762781


In [110]:
Y_test_hat = np.argmax(model.predict(X),axis=1)

In [111]:
Y_test_hat

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [113]:
df_test['prediction'] = Y_test_hat

In [118]:
results = df_train[['id','prediction']]

In [120]:
results.to_csv('temp_team.csv')

In [124]:
df_train[['id','review','cleaned_text']].to_csv('cleaned_training.csv')

In [6]:
plot_model(model, to_file="model.png", show_shapes=True)

In [22]:
import pickle
f = open("Data/train_test_data.pkl", 'rb')

In [23]:
data = pickle.load(f)

In [35]:
X_train = data["X_train"]
y_train = data["y_train"]
X_test = data["X_test"]
y_test = data["y_test"]

## TextBlob

In [40]:
from textblob import TextBlob

In [56]:
def get_polarity(review):
    polarity = TextBlob(review).sentiment.polarity
    if polarity > 0:
        return 1
    else:
        return 0

In [69]:
polarity_pred = analyzer.polarity_scores(cleaned_train['review'][3])

In [81]:
polarity_pred_blob = [get_polarity(review) for review in cleaned_train['review']]

In [80]:
y_pred_blob = polarity_pred

In [70]:
y_true = cleaned_train['sentiment']

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, polarity_pred)

0.69675834191963226

In [61]:
y_true.shape

(7860,)

In [63]:
df_train.keys()

Index(['id', 'review'], dtype='object')

In [68]:
cleaned_train = pd.read_csv('Data/cleaned_train.csv')

## Vader sentiment

In [73]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [79]:
analyzer = SentimentIntensityAnalyzer()

{'compound': 0.9454, 'neg': 0.061, 'neu': 0.776, 'pos': 0.163}

In [82]:
def get_compound(review):
    compound = analyzer.polarity_scores(review)['compound']
    if compound > 0:
        return 1
    else:
        return 0

In [83]:
polarity_pred_analyzer = [get_compound(review) for review in cleaned_train['review']]

In [84]:
y_pred_vader = polarity_pred_analyzer

In [85]:
accuracy_score(y_true, y_pred_vader)

0.69081398113656178